In [11]:
import pandas as pd
import os
import numpy as np

"""windows  
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM\\1_cluster_demand_prediction\data')
"""

""" LInux """
os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/2_freq_nbinom_LSTM/1_cluster_demand_prediction/data/demand_data/New_BlueSG_clusters")
!pwd

region_name = "tampines"

""" windows 
df_dem = pd.read_csv('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM\\1_cluster_demand_prediction\data\demand_data\\' + region_name + '_region_demand2.csv')
df_inf = pd.read_csv('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM\\1_cluster_demand_prediction\data\demand_data\\' + region_name + '_region_inflow2.csv')
"""

""" Linux """
df_dem = pd.read_csv( region_name + '_region_outflow.csv')
df_inf = pd.read_csv( region_name + '_region_inflow.csv')


cov_lag_len = 1


df_dem = pd.DataFrame(df_dem.iloc[cov_lag_len:]).reset_index(drop=True)
df_inf = pd.DataFrame(df_inf.iloc[:-cov_lag_len]).reset_index(drop=True)

min_idx = min(len(df_dem)-1, len(df_inf)-1) # this is bcos pandas includes last index as well
df_dem = df_dem.loc[:min_idx]
df_inf = df_inf.loc[:min_idx]

df_dem_lag_168 = df_dem[:-168].reset_index()
df_dem_lag_336 = df_dem[:-336].reset_index()

df_inf_lag_168 = df_inf[:-168].reset_index()
df_inf_lag_336 = df_inf[:-336].reset_index()

df_dem = df_dem[584:].reset_index() # make it a medium dataset
df_inf = df_inf[584:].reset_index() # make it a medium dataset


df_dem_lag_168 = df_dem_lag_168[416:].reset_index() # make it a medium dataset
df_inf_lag_168 = df_inf_lag_168[416:].reset_index() # make it a medium dataset

df_dem_lag_336 = df_dem_lag_336[248:].reset_index() # make it a medium dataset
df_inf_lag_336 = df_inf_lag_336[248:].reset_index() # make it a medium dataset


# df_dem_lag_168 = df_dem_lag_168[168:].reset_index()
# df_dem = df_dem[336:].reset_index()

# df_inf_lag_168 = df_inf_lag_168[168:].reset_index()
# df_inf = df_inf[336:].reset_index()


df_dem = df_dem.drop(["index"],axis=1)
df_inf = df_inf.drop(["index"],axis=1)
df_dem_lag_168 = df_dem_lag_168.drop(["index", "level_0"],axis=1)
df_inf_lag_168 = df_inf_lag_168.drop(["index", "level_0"],axis=1)
df_dem_lag_336 = df_dem_lag_336.drop(["index"],axis=1)
df_inf_lag_336 = df_inf_lag_336.drop(["index"],axis=1)



# print(len(df_dem))
# print(len(df_inf))

#new_df = pd.DataFrame(columns=["target","group","inflow","dem_lag_168","dem_lag_336","inf_lag_168","inf_lag_336" ])

#arr = np.empty((0,7), int)
train_arr = np.empty((0,8), int)
val_arr = np.empty((0,8), int)
test_arr = np.empty((0,8), int)

g=0
for c in df_dem.columns:

    s = pd.Series(np.repeat(g,len(df_inf)))
    df_grp = pd.DataFrame({c:s})
    df_time_idx = pd.DataFrame(np.arange(0,len(df_inf),1))

    df_concat = pd.concat([df_time_idx, df_dem[c], df_grp, df_inf[c], df_dem_lag_168[c],df_dem_lag_336[c], df_inf_lag_168[c], df_inf_lag_336[c] ],axis=1 )
    df_concat = df_concat.to_numpy()
    len_df = df_concat.shape[0]


    #arr = np.append(arr, df_concat,axis=0) 
    tr_stop_idx = int(0.7*len_df)
    val_stop_idx = (int(0.7*len_df) + int(0.2*len_df))
    # print(tr_stop_idx)
    # print(val_stop_idx)

    train_arr = np.append(train_arr, df_concat[:tr_stop_idx],axis=0)
    val_arr = np.append(val_arr, df_concat[tr_stop_idx:val_stop_idx ],axis=0)
    test_arr = np.append(test_arr, df_concat[val_stop_idx:],axis=0)

    g+=1

# new_df = pd.DataFrame(arr, columns=["target","group","inflow","dem_lag_168","dem_lag_336","inf_lag_168","inf_lag_336" ])
# new_df['time_idx'] = np.arange(0,len(new_df),1)
# first_column = new_df.pop('time_idx')
# new_df.insert(0, 'time_idx', first_column)


new_train_df = pd.DataFrame(train_arr, columns=["time_idx","target","group","inflow","dem_lag_168","dem_lag_336","inf_lag_168","inf_lag_336" ])
# first_column = new_train_df.pop('time_idx')
# new_train_df.insert(0, 'time_idx', first_column)


new_val_df = pd.DataFrame(val_arr, columns=["time_idx","target","group","inflow","dem_lag_168","dem_lag_336","inf_lag_168","inf_lag_336" ])
# first_column = new_val_df.pop('time_idx')
# new_val_df.insert(0, 'time_idx', first_column)


new_test_df = pd.DataFrame(test_arr, columns=["time_idx","target","group","inflow","dem_lag_168","dem_lag_336","inf_lag_168","inf_lag_336" ])
# first_column = new_test_df.pop('time_idx')
# new_test_df.insert(0, 'time_idx', first_column)


#print(new_df)

"""windows 
os.chdir("C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\\2_freq_nbinom_LSTM\\1_cluster_demand_prediction\data\demand_data")
"""

""" LInux """
# os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/2_freq_nbinom_LSTM/1_cluster_demand_prediction/data/demand_data/New_BlueSG_clusers")

#new_df.to_csv('tampines_all_clstr_dem_data.csv')
new_train_df.to_csv(region_name+'_all_clstr_train_dem_data.csv')
new_val_df.to_csv(region_name+'_all_clstr_val_dem_data.csv')
new_test_df.to_csv(region_name+'_all_clstr_test_dem_data.csv')



"""

Need to adjust the dataset for this.

TODO: Need to handle cov_lag_len here.


"""





/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/2_freq_nbinom_LSTM/1_cluster_demand_prediction/data/demand_data/New_BlueSG_clusters


'\n\nNeed to adjust the dataset for this.\n\nTODO: Need to handle cov_lag_len here.\n\n\n'

In [12]:
col_list = new_test_df.columns

for c in col_list:
    print(c,new_test_df[c].isnull().values.any())

time_idx False
target False
group False
inflow False
dem_lag_168 False
dem_lag_336 False
inf_lag_168 False
inf_lag_336 False


In [13]:
df_inf

,160,161,162,163,164,165,166,167,168,169,170
0,1,0,1,1,0,1,0,1,4,1,0
1,1,0,3,0,1,1,0,0,0,0,1
2,2,0,2,0,0,3,1,1,1,0,1
3,0,0,2,1,0,1,0,2,3,0,0
4,1,0,1,0,1,0,0,1,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...
1590,1,1,0,0,1,1,0,3,5,2,1
1591,0,1,2,0,0,1,0,0,2,1,1
1592,1,1,3,0,1,5,4,0,2,2,1
1593,1,2,1,0,0,0,1,0,1,2,2
